In [1]:
import os

In [9]:
%pwd

'/Users/abdulrasheed/Desktop/ImageClassification'

In [10]:
os.chdir('../'
)

In [11]:
%pwd

'/Users/abdulrasheed/Desktop'

In [12]:
cd ImageClassification

/Users/abdulrasheed/Desktop/ImageClassification


In [13]:
from pathlib import Path
from dataclasses import dataclass

@dataclass(frozen=True)
class TrainingConfig:
    root_dir : Path
    trained_model_path : Path
    updated_base_model_path: Path
    training_data : Path
    params_epochs : int
    params_batch_size : int
    params_is_augmentation : bool
    params_image_size : list

@dataclass(frozen=True)
class PrepareCallbacksConfig:
    root_dir : Path
    tensorboard_root_log_dir : Path
    checkpoint_model_filepath : Path
    

In [14]:
from source.CNNClassifier.constant.__init_ import CONFIG_FILE_PATH, PARAMS_FILE_PATH
from source.CNNClassifier.utils.common import read_yaml, create_directories
import tensorflow as tf

In [15]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])


    def get_prepare_callback_config(self) -> PrepareCallbacksConfig:
        config = self.config.prepare_callbacks
        model_ckpt_dir = os.path.dirname(config.checkpoint_model_filepath)
        create_directories([
            Path(model_ckpt_dir),
            Path(config.tensorboard_root_log_dir)
        ])

        prepare_callback_config = PrepareCallbacksConfig(
            root_dir=Path(config.root_dir),
            tensorboard_root_log_dir=Path(config.tensorboard_root_log_dir),
            checkpoint_model_filepath=Path(config.checkpoint_model_filepath)
        )

        return prepare_callback_config
    

    def get_training_config(self) -> TrainingConfig:
        training = self.config.training
        prepare_base_model = self.config.prepare_base_model
        params = self.params
        training_data = os.path.join(self.config.data_ingestion.unzip_dir, "Images")
        print(training_data)
        create_directories([
            Path(training.root_dir)
        ])

        training_config = TrainingConfig(
            root_dir=Path(training.root_dir),
            trained_model_path=Path(training.training_model_path),
            updated_base_model_path=Path(prepare_base_model.updated_base_model_path),
            training_data=Path(training_data),
            params_epochs=params.EPOCHS,
            params_batch_size=params.BATCH_SIZE,
            params_is_augmentation=params.AUGMENTATION,
            params_image_size=params.IMAGE_SIZE
        )

        return training_config


In [16]:
import time

In [17]:
class PrepareCallback:
    def __init__(self, config: PrepareCallbacksConfig):
        self.config = config


    
    @property
    def _create_tb_callbacks(self):
        timestamp = time.strftime("%Y-%m-%d-%H-%M-%S")
        tb_running_log_dir = os.path.join(
            self.config.tensorboard_root_log_dir,
            f"tb_logs_at_{timestamp}",
        )
        return tf.keras.callbacks.TensorBoard(log_dir=tb_running_log_dir)
    

    @property
    def _create_ckpt_callbacks(self):
        return tf.keras.callbacks.ModelCheckpoint(
            filepath=self.config.checkpoint_model_filepath,
            save_best_only=True
        )


    def get_tb_ckpt_callbacks(self):
        return [
            self._create_tb_callbacks,
            self._create_ckpt_callbacks
        ]

In [18]:
import os
import urllib.request as request
from zipfile import ZipFile
import tensorflow as tf
import time
from PIL import Image
from PIL import UnidentifiedImageError

In [19]:
! pip install Pillow



In [ ]:


class Training:
    def __init__(self, config: TrainingConfig):
        self.config = config

    def get_base_model(self):
        self.model = tf.keras.models.load_model(
            self.config.updated_base_model_path
        )

    def remove_ds_store_files(self, directory):
        """
        Remove all .DS_Store files from the specified directory and its subdirectories.
        """
        for root, _, files in os.walk(directory):
            if '.DS_Store' in files:
                os.remove(os.path.join(root, '.DS_Store'))
                print(f".DS_Store has been removed from: {root}")

    def remove_invalid_images(self, directory):
        """
        Validate and remove corrupted image files from the specified directory and its subdirectories.
        """
        for root, _, files in os.walk(directory):
            for file in files:
                file_path = os.path.join(root, file)
                try:
                    with Image.open(file_path) as img:
                        img.verify()  # Verify that it's a valid image
                except (IOError, SyntaxError, Image.UnidentifiedImageError):
                    print(f"Removing corrupted file: {file_path}")
                    os.remove(file_path)

    def train_valid_generator(self):
        # Clean the training data directory
        training_data_dir = self.config.training_data
        print("Cleaning the training data directory...")
        self.remove_ds_store_files(training_data_dir)
        self.remove_invalid_images(training_data_dir)

        # Common data generator arguments
        datagenerator_kwargs = dict(
            rescale=1. / 255,
            validation_split=0.20
        )

        dataflow_kwargs = dict(
            target_size=self.config.params_image_size[:-1],
            batch_size=self.config.params_batch_size,
            interpolation="bilinear"
        )

        # Validation data generator
        valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenerator_kwargs
        )

        self.valid_generator = valid_datagenerator.flow_from_directory(
            directory=training_data_dir,
            subset="validation",
            shuffle=False,
            **dataflow_kwargs
        )

        # Training data generator
        if self.config.params_is_augmentation:
            train_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
                rotation_range=40,
                horizontal_flip=True,
                width_shift_range=0.2,
                height_shift_range=0.2,
                shear_range=0.2,
                zoom_range=0.2,
                **datagenerator_kwargs
            )
        else:
            train_datagenerator = valid_datagenerator

        self.train_generator = train_datagenerator.flow_from_directory(
            directory=training_data_dir,
            subset="training",
            shuffle=True,
            **dataflow_kwargs
        )

    @staticmethod
    def save_model(path: Path, model: tf.keras.Model):
        model.save(path)

    def train(self, callback_list: list):
        self.steps_per_epoch = self.train_generator.samples // self.train_generator.batch_size
        self.validation_steps = self.valid_generator.samples // self.valid_generator.batch_size

        self.model.fit(
            self.train_generator,
            epochs=self.config.params_epochs,
            steps_per_epoch=self.steps_per_epoch,
            validation_steps=self.validation_steps,
            validation_data=self.valid_generator,
            callbacks=callback_list
        )

        self.save_model(
            path=self.config.trained_model_path,
            model=self.model
        )


In [44]:
try:
    config = ConfigurationManager()
    prepare_callbacks_config = config.get_prepare_callback_config()
    prepare_callbacks = PrepareCallback(config=prepare_callbacks_config)
    callback_list = prepare_callbacks.get_tb_ckpt_callbacks()

    training_config = config.get_training_config()
    training = Training(config=training_config)
    training.get_base_model()
    training.train_valid_generator()
    training.train(
        callback_list=callback_list
    )
    
except Exception as e:
    raise e

[2024-12-16 15:18:11,113: INFO: common: yaml file: config/config.yaml loaded successfully]
[2024-12-16 15:18:11,116: INFO: common: yaml file: params.yaml loaded successfully]
[2024-12-16 15:18:11,117: INFO: common: created directory at: artifacts]
[2024-12-16 15:18:11,118: INFO: common: created directory at: artifacts/prepare_callbacks/checkpoint_dir]
[2024-12-16 15:18:11,119: INFO: common: created directory at: artifacts/prepare_callbacks/tensorboard_log_dir]
artifacts/data_ingestion/Images
[2024-12-16 15:18:11,121: INFO: common: created directory at: artifacts/training]
Cleaning the training data directory...
Removing corrupted file: artifacts/data_ingestion/Images/cat/666.jpg
Found 2166 images belonging to 4 classes.
Found 8673 images belonging to 4 classes.
542/542 ━━━━━━━━━━━━━━━━━━━━ 1317s 2s/step - accuracy: 0.3539 - loss: 15.3982 - val_accuracy: 0.4093 - val_loss: 15.2092
